In [1]:
from src.returns_analysis import analyse_returns_characteristics
from src.process_selection import process_selection, recommend_process
from src.var import produce_var_results
from src.processes import Processes
from src.get_data import HistoricalData
from src.util import parse_json
import plotly.graph_objects as go
import numpy as np

In [2]:
config = parse_json("config.json")

histdata = HistoricalData(config["binance"]["API_KEY"], config["binance"]["API_SECRET"])
df = histdata.getKline("SOLUSDC","1h").dropna()
# histdata.save_data("test_data.csv")

In [3]:
# df = pd.read_csv("test_data.csv").dropna()
# df.head()

In [4]:
stats_summary = analyse_returns_characteristics(df)

BASIC STATS
mean : -0.0001419977250655984
std : 0.007575785463387278
Sharpe Ratio : -0.018743630710221896
skewness : -0.4368650065328801
kurtosis : 5.866399556930414
jb_pvalue : 0.0
min : -0.045033611584007316
max : 0.04191262091984828


In [5]:
test_results = process_selection(df)

In [6]:
recommend_process(test_results,stats_summary)

[{'process': 'jump diffusion (merton)',
  'confidence': 'medium',
  'reason': 'significant jump frequency'},
 {'process': 'Ornstein-Uhlenbeck',
  'confidence': 'medium',
  'reason': 'evidence of mean reversion in returns'},
 {'process': 'heston stochastic volatility',
  'confidence': 'medium',
  'reason': 'non-normal distrib with fat tails and skewness'}]

In [7]:
processes = Processes(df)

GBM

In [8]:
price_path = processes.gbm_price_path(24*60,1_000)
produce_var_results(price_path,df)

Jump Diffusion Process

In [9]:
price_path = processes.jdp(24*60,1_000)
produce_var_results(price_path,df)

OU Process

In [10]:
price_path = processes.ou(24*60,1_000)
produce_var_results(price_path,df)

/home/o/Projects/Monte-Carlo-VaR/src/processes.py:273: RuntimeWarning:

overflow encountered in power

